# **Khushdev Pandit**
# **Roll no: 2020211** 
# *Assignment Question-2*

#

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import open3d as o3d
import glob

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# **Q2 Part-1**
* ##### (10 points) Compute the chessboard plane normals nLi, i ∈ {1, . . . , 25} and the corresponding offsets di , i ∈ {1, . . . , 25} using the planar LIDAR points in each of the pcd files. You can estimate these by making use of singular value decomposition.

In [ ]:
def load_camera_parameters(camera_matrix_path, distortion_coefficients_path):
    with open(distortion_coefficients_path, 'r') as f:
        contents = f.read()
        lines = contents.split('\n')[:1]
        lines = np.array(lines, dtype=np.float32)
        print(lines)
    pass

In [2]:
lidar_scans = glob.glob('lidar_scans/*.pcd')
camera_images = glob.glob('camera_images/*.jpeg')
camera_param = glob.glob('camera_images/*.jpeg')
indices = np.arange(0, len(lidar_scans))
indices = np.random.choice(indices, 25, replace=False)

In [3]:
camera_images, lidar_scans, camera_param = np.array(camera_images), np.array(lidar_scans), np.array(camera_param)
camera_images, lidar_scans, camera_param = camera_images[indices], lidar_scans[indices], camera_param[indices]

In [91]:
normal_open3d = []
offset_open3d = []

for lidar in lidar_scans:
    pcd = o3d.io.read_point_cloud(lidar)
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)
    normal = plane_model[:3]
    offset = plane_model[3]
    normal_open3d.append(normal)
    offset_open3d.append(offset)
normal_open3d, offset_open3d = np.array(normal_open3d), np.array(offset_open3d)

In [93]:
all_normals = []
all_offsets = []

for lidar in lidar_scans:
    pcd = o3d.io.read_point_cloud(lidar)
    # Estimate plane model coefficients using RANSAC algorithm
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)

    # Extract inlier points
    points = np.asarray(pcd.points)
    # Compute centroid of the planar points
    centroid = np.mean(points, axis=0)
    # Subtract centroid from the points
    centered_points = points - centroid
    # Perform SVD on the centered points
    __, _, vh = np.linalg.svd(centered_points)
    # Estimate plane normal
    normal = vh[-1, :]  # Last row of vh matrix
    # Estimate offset of the plane
    offset = np.dot(normal, centroid)
    all_normals.append(normal)
    all_offsets.append(offset)

all_normals, all_offsets = np.array(all_normals), np.array(all_offsets)

In [94]:
res = normal_open3d - all_normals
res.mean(), res.std()

(0.18287762487068762, 0.8123856214511134)

In [96]:
# Open the text file in read mode
with open('camera_parameters/frame_256.jpeg/camera_normals.txt', 'r') as file:
    contents = file.read()
    lines = contents.split('\n')[:3]
    lines = np.array(lines, dtype=np.float32)
    print(lines)


[ 0.56540185 -0.19615303 -0.80115217]
[[0.5654018521308899], [-0.19615302979946136], [-0.8011521697044373]]
